# Основы программирования в Python

*Автор: Анастасия Паршина, НИУ ВШЭ*  

## Web-scraping: скрэйпинг сайта с книжками

In [2]:
import requests
from bs4 import BeautifulSoup 
from time import sleep
import pandas as pd

Дана ссылка на первую страницу [каталога книжных новинок](http://www.biblio-globus.ru/search/catalog/products?query=Python&page=1). Соберите информацию о книгах с первых пяти страниц. Если присмотреться, то ссылки на страницы выгядят похоже: 

    http://www.biblio-globus.ru/search/catalog/products?query=Python&page=1
    http://www.biblio-globus.ru/search/catalog/products?query=Python&page=2
    http://www.biblio-globus.ru/search/catalog/products?query=Python&page=3
    
Значит, мы можем сгенерировать необходимые ссылки сами. В таком случае чтобы узнать конечную страницу, нужно будет не нее перейти и посмотреть номер. Что, конечно не супер автоматизировано, но так мы точно не ошибемся. 

In [3]:
urls = []

for i in range(1, 6):
    urls.append(f'http://www.biblio-globus.ru/search/catalog/products?query=Python&page={i}')
    
urls

['http://www.biblio-globus.ru/search/catalog/products?query=Python&page=1',
 'http://www.biblio-globus.ru/search/catalog/products?query=Python&page=2',
 'http://www.biblio-globus.ru/search/catalog/products?query=Python&page=3',
 'http://www.biblio-globus.ru/search/catalog/products?query=Python&page=4',
 'http://www.biblio-globus.ru/search/catalog/products?query=Python&page=5']

Другой вариант действий: в коде страницы есть ссыка на "Следующую" страницу. Можно собирать именно их *до тех пор, пока* они не закончатся (тут вам в помощь цикл `while`).

Разберем все на примере одной страницы. 

In [4]:
page = requests.get(urls[0])
soup = BeautifulSoup(page.text)

Заберем ссылки на все книжки на странице. 

In [5]:
urls_books = ['http://www.biblio-globus.ru' + i.get('href') \
              for i in soup.find_all('a', attrs = {'class':'name'})]
urls_books

['http://www.biblio-globus.ru/search/catalog/details/10839595',
 'http://www.biblio-globus.ru/search/catalog/details/10841699',
 'http://www.biblio-globus.ru/search/catalog/details/10224348',
 'http://www.biblio-globus.ru/search/catalog/details/10776656',
 'http://www.biblio-globus.ru/search/catalog/details/10483871',
 'http://www.biblio-globus.ru/search/catalog/details/10577094',
 'http://www.biblio-globus.ru/search/catalog/details/10569801',
 'http://www.biblio-globus.ru/search/catalog/details/10835402',
 'http://www.biblio-globus.ru/search/catalog/details/10788303',
 'http://www.biblio-globus.ru/search/catalog/details/10829190']

Посмотрим на одну книжку: 

In [6]:
url0 = urls_books[0]
page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text)

Найдем основную информацию со страницы

In [7]:
author = soup0.find_all('div', attrs = {'class' : 'details_author'})[0].text
author

'Голиков Д. В.'

Выше мы ищем тег `div`, при этом уточним, что у этого тега есть атрибут `class` со значением `details_author`.

In [8]:
title = soup0.find_all('div', attrs = {'class' : 'details_name'})[0].text
title

'Python для юных программистов'

Обратим внимание на статус - "В наличии". Кусочек кода выглядит так:

    <div class="title_data green_card" style="font-size:12px !important;">В наличии</div>
    
НО! Есть книжки, которых нет в наличии. И у них эта же часть выглядит так: 

    <div class="title_data grey_card" style="font-size:12px !important;">Под заказ</div>
    
В таком случае у атрибута `class` два значения. Нас интересует то, которое универсальное, то есть `title_data`.

In [9]:
status = soup0.find_all('div', attrs = {'class' : 'title_data'})[0].text.strip()
status

'В наличии'

Можно также посмотреть на расположение книжки в магазине (опять же в коде нужно будет потом учитывать, что не все книжки есть в наличии).

In [10]:
L = soup0.find_all('ul')[0].text.split('\n')
L

['', 'Зал № 7', 'секция: 8', 'шкаф: 76', 'полка: 6', 'уровень: 1', '']

In [12]:
Data = {}

for i in L:
    if len(i) != 0:
        if ':' in i:
            Data[i.split(':')[0]] = i.split(':')[1].strip()
        else:
            Data[i.split()[0].lower()] = i.split()[2]
Data

{'зал': '7', 'секция': '8', 'шкаф': '76', 'полка': '6', 'уровень': '1'}

Можно также попробовать вытащить аннотацию.

In [13]:
ann = soup0.find_all('div', attrs = {'class' : 'details_additional_info'})[0].text
ann

'Книга написана на основе опыта обучения программированию на языке Python в кружке юных программистов и протестирована на детях 7–12 лет. Материал рассчитан на самостоятельное, без помощи взрослых, изучение основ Python школьниками 2–5 классов, имеющими базовые навыки управления компьютером. Процесс создания программ дан пошагово со скриншотами. При создании первых программ дети не пишут код, а собирают программу из разноцветных блоков, как в Scratch, затем превращают ее в код и вносят в него небольшие правки. Дети научатся программировать черепашку для рисования узоров, а также создавать постройки, управлять персонажами и дронами в Minecraft. В процессе игрового обучения дети узнают о типах данных, операторах, циклах, списках, функциях и других элементах языка Python.'

Вроде бы все хорошо, однако на паре мы наткнулись на случаи, когда по такому запросу `'div', attrs = {'class' : 'details_additional_info'}` выдается не аннотация, а любая другая дополнительная информация. Поэтому в итоговом коде мы добавим условие на длину выдачи (считаем, что в аннотации больше 50 смиволов). 

In [14]:
def GetBookInfo(url0):
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text)
    
    author = soup0.find_all('div', {'class':'details_author'}) # а еще у нас автор может быть указан не всегда
    
    if len(author) == 0:
        author = 'Автора нет'
    else:
        author = author[0].text
        
    name = soup0.find_all('div', {'class':'details_name'})[0].text
    status = soup0.find_all('div', {'class' : 'title_data'})[0].text.strip()
    price = float(soup0.find_all('div', {'class':'details_price'})[0].text.strip().split()[0].replace(',', '.'))
    
    if status == 'В наличии':
        L = soup0.find_all('ul')[0].text.split('\n')
        Data = {}

        for i in L:
            if len(i) != 0:
                if ':' in i:
                    Data[i.split(':')[0]] = i.split(':')[1].strip()
                else:
                    Data[i.split()[0].lower()] = i.split()[2]
                    
    else:
        Data = 'Нет в наличии'
    
    ann = soup0.find_all('div', attrs = {'class' : 'details_additional_info'})
    
    for i in ann:
        if len(i.text) < 50:
            pass
        else:
            ann = i.text
        
    return url0, author, name, status, price, Data, ann

Пробуем нашу функцию применить. 

In [15]:
books = []
for u in urls[:5]:
    page0 = requests.get(u)
    soup0 = BeautifulSoup(page0.text)
    sleep(1)
    books_urls = ['http://www.biblio-globus.ru' + i.get('href') \
                  for i in soup0.find_all('a', {'class':'name'})]
    
    for i in books_urls:
        print(i) # печатаем ссылки, что просто понимать, что у нас ничего не сломалось и все работает 
        res = GetBookInfo(i)
        books.append(res)
        sleep(1)

http://www.biblio-globus.ru/search/catalog/details/10839595
http://www.biblio-globus.ru/search/catalog/details/10841699
http://www.biblio-globus.ru/search/catalog/details/10224348
http://www.biblio-globus.ru/search/catalog/details/10776656
http://www.biblio-globus.ru/search/catalog/details/10483871
http://www.biblio-globus.ru/search/catalog/details/10577094
http://www.biblio-globus.ru/search/catalog/details/10569801
http://www.biblio-globus.ru/search/catalog/details/10835402
http://www.biblio-globus.ru/search/catalog/details/10788303
http://www.biblio-globus.ru/search/catalog/details/10829190
http://www.biblio-globus.ru/search/catalog/details/10823441
http://www.biblio-globus.ru/search/catalog/details/10410028
http://www.biblio-globus.ru/search/catalog/details/10831874
http://www.biblio-globus.ru/search/catalog/details/10829080
http://www.biblio-globus.ru/search/catalog/details/10825928
http://www.biblio-globus.ru/search/catalog/details/10822253
http://www.biblio-globus.ru/search/catal

In [17]:
df = pd.DataFrame(books)
df.columns = ['url', 'author', 'title', 'status', 'price', 'place', 'ann']
df.head(4)

,url,author,title,status,price,place,ann
0,http://www.biblio-globus.ru/search/catalog/det...,Голиков Д. В.,Python для юных программистов,В наличии,1059.0,"{'зал': '7', 'секция': '8', 'шкаф': '76', 'пол...",Книга написана на основе опыта обучения програ...
1,http://www.biblio-globus.ru/search/catalog/det...,Борзунов С. В.,Алгебра и геометрия с примерами на Python,В наличии,4689.0,"{'зал': '7', 'секция': '8', 'шкаф': '76', 'пол...",Учебное пособие является современным введением...
2,http://www.biblio-globus.ru/search/catalog/det...,МакГрат М.,Программирование на Python для начинающих,В наличии,789.0,"{'зал': '7', 'секция': '8', 'шкаф': '76', 'пол...","Книга ""Программирование на Python для начинаю..."
3,http://www.biblio-globus.ru/search/catalog/det...,Кольцов Д. М.,"Справочник PYTHON. Кратко, быстро, под рукой",В наличии,499.0,"{'зал': '7', 'секция': '8', 'шкаф': '76', 'пол...",Данный справочник содержит всю ключевую информ...


In [18]:
df.to_csv('books.csv')